In [1]:
import os

# Specify estimand
trial_pp = {"estimand": "PP"}  # Per-protocol
trial_itt = {"estimand": "ITT"}  # Intention-to-treat

# Create directories to save files
trial_pp_dir = os.path.join(os.getcwd(), "trial_pp")
os.makedirs(trial_pp_dir, exist_ok=True)

trial_itt_dir = os.path.join(os.getcwd(), "trial_itt")
os.makedirs(trial_itt_dir, exist_ok=True)

import pandas as pd

# Load dummy data (assuming you have a CSV file or similar)
data_censored = pd.read_csv("../data/data_censored.csv")

# Display the first few rows of the data
print(data_censored.head())

# Per-protocol
trial_pp["data"] = data_censored
trial_pp["id"] = "id"
trial_pp["period"] = "period"
trial_pp["treatment"] = "treatment"
trial_pp["outcome"] = "outcome"
trial_pp["eligible"] = "eligible"

# Intention-to-treat
trial_itt["data"] = data_censored
trial_itt["id"] = "id"
trial_itt["period"] = "period"
trial_itt["treatment"] = "treatment"
trial_itt["outcome"] = "outcome"
trial_itt["eligible"] = "eligible"

import statsmodels.api as sm
from statsmodels.formula.api import logit

# Censoring due to treatment switching (Per-protocol)
numerator_formula_pp = "treatment ~ age"
denominator_formula_pp = "treatment ~ age + x1 + x3"

# Fit numerator and denominator models
numerator_model_pp = logit(numerator_formula_pp, data=data_censored).fit()
denominator_model_pp = logit(denominator_formula_pp, data=data_censored).fit()

# Save models to disk (optional)
numerator_model_pp.save(os.path.join(trial_pp_dir, "numerator_model_pp.pkl"))
denominator_model_pp.save(os.path.join(trial_pp_dir, "denominator_model_pp.pkl"))

# Other informative censoring (Per-protocol and ITT)
censor_event = "censored"
numerator_formula_censor = "1 - censored ~ x2"
denominator_formula_censor = "1 - censored ~ x2 + x1"

# Fit models for Per-protocol
numerator_model_censor_pp = logit(numerator_formula_censor, data=data_censored).fit()
denominator_model_censor_pp = logit(denominator_formula_censor, data=data_censored).fit()

# Fit models for ITT
numerator_model_censor_itt = logit(numerator_formula_censor, data=data_censored).fit()
denominator_model_censor_itt = logit(denominator_formula_censor, data=data_censored).fit()

# Save models to disk (optional)
numerator_model_censor_pp.save(os.path.join(trial_pp_dir, "numerator_model_censor_pp.pkl"))
denominator_model_censor_pp.save(os.path.join(trial_pp_dir, "denominator_model_censor_pp.pkl"))
numerator_model_censor_itt.save(os.path.join(trial_itt_dir, "numerator_model_censor_itt.pkl"))
denominator_model_censor_itt.save(os.path.join(trial_itt_dir, "denominator_model_censor_itt.pkl"))

# Calculate weights for Per-protocol
data_censored["numerator_weight_pp"] = numerator_model_pp.predict(data_censored)
data_censored["denominator_weight_pp"] = denominator_model_pp.predict(data_censored)
data_censored["weight_pp"] = data_censored["numerator_weight_pp"] / data_censored["denominator_weight_pp"]

# Calculate weights for ITT
data_censored["numerator_weight_itt"] = numerator_model_censor_itt.predict(data_censored)
data_censored["denominator_weight_itt"] = denominator_model_censor_itt.predict(data_censored)
data_censored["weight_itt"] = data_censored["numerator_weight_itt"] / data_censored["denominator_weight_itt"]

# Specify outcome model for Per-protocol
outcome_formula_pp = "outcome ~ treatment + x2"
outcome_model_pp = logit(outcome_formula_pp, data=data_censored).fit()

# Specify outcome model for ITT with adjustment terms
outcome_formula_itt = "outcome ~ treatment + x2"
outcome_model_itt = logit(outcome_formula_itt, data=data_censored).fit()

# Expand trials for Per-protocol
trial_pp["expanded_data"] = data_censored.copy()
trial_pp["expanded_data"]["trial_period"] = 0  # Example: set trial period to 0

# Expand trials for ITT
trial_itt["expanded_data"] = data_censored.copy()
trial_itt["expanded_data"]["trial_period"] = 0  # Example: set trial period to 0

# Sample data for ITT
sampled_data_itt = data_censored.sample(frac=0.5, random_state=1234)

# Fit marginal structural model for ITT
msm_formula_itt = "outcome ~ treatment + x2"
msm_model_itt = logit(msm_formula_itt, data=sampled_data_itt).fit()

# Print model summary
print(msm_model_itt.summary())

# Predict survival probabilities for ITT
new_data = sampled_data_itt[sampled_data_itt["trial_period"] == 1]
predict_times = range(0, 11)
predictions = msm_model_itt.predict(new_data)

# Plot survival differences (example)
import matplotlib.pyplot as plt

plt.plot(predict_times, predictions, label="Survival Difference")
plt.xlabel("Follow up")
plt.ylabel("Survival difference")
plt.legend()
plt.show()

   id  period  treatment  x1        x2  x3        x4  age     age_s  outcome  \
0   1       0          1   1  1.146148   0  0.734203   36  0.083333        0   
1   1       1          1   1  0.002200   0  0.734203   37  0.166667        0   
2   1       2          1   0 -0.481762   0  0.734203   38  0.250000        0   
3   1       3          1   0  0.007872   0  0.734203   39  0.333333        0   
4   1       4          1   1  0.216054   0  0.734203   40  0.416667        0   

   censored  eligible  
0         0         1  
1         0         0  
2         0         0  
3         0         0  
4         0         0  


ModuleNotFoundError: No module named 'statsmodels'